In [1]:
import torch
from torch.utils.data import Dataset, DataLoader
import torchvision
import time
import torch
import torch.nn as nn
import torchvision.transforms as transforms
from torchvision import utils as vutils
import numpy as np

In [2]:
# %%
import datetime
import os
import time
from pathlib import Path
from typing import Dict, List, Optional, Union

import matplotlib.pyplot as plt
import numpy as np
import torch
import torch.nn as nn
import torchvision.transforms as transforms
from torch.utils.data import DataLoader, Dataset
from torchvision import utils as vutils


class MyDataset(Dataset):
    def __init__(self, dir_list, transform=None):
        self.dir_list = dir_list
        self.transform = transform

        self.datas, self.labels = self.load_data(dir_list)

    def load_data(self, class_list):
        datas = []
        labels = []
        for name in class_list:
            data = np.load(f'dataset\\{name}\\data.npy', allow_pickle=True)
            
            if name == 'normal':
                label = np.ones(len(data))
            else:
                label = np.zeros(len(data))
            if len(datas) == 0:
                datas = data
                labels = label
            else:

                datas = np.concatenate([datas, data],axis=0)
                labels = np.concatenate([labels, label])
        datas = np.transpose(datas, (0, 2, 1))  
        
        max_list = np.amax(datas, axis=2)
        max = np.amax(max_list, axis=0)
        print(max)
        max_x = max[0]
        max_y = max[1]
        datas[:][0] = datas[:][0]/max_x
        datas[:][1] = datas[:][1]/max_y

        #print(datas)
        
        
        return datas, labels
    def __len__(self):
        return len(self.datas)

    def __getitem__(self, idx):

        data = self.datas[idx]
        label = self.labels[idx]
        if self.transform:
            data = self.transform(data)

        
        return data, label


In [48]:
transforms_ = transforms.Compose([
    transforms.ToTensor(),
    transforms.Normalize([0.5], [0.5])
    ])
data_list = ['normal', 'bad_leg']
mydataset = MyDataset(data_list, transforms_)
train_len = int(len(mydataset)*0.8)
test_len = len(mydataset) - train_len
train_set, val_set = torch.utils.data.random_split(mydataset, [train_len, test_len])

[644.0021    449.46243     0.9781028]


In [51]:
len(train_set)

1129

In [5]:
class Net(nn.Module):
    def __init__(self):
        super(Net, self).__init__()
        self.fc1 = nn.Linear(in_features=75, out_features=256)
        self.fc2 = nn.Linear(in_features=256, out_features=256)
        self.fc3 = nn.Linear(in_features=256, out_features=64)
        self.fc4 = nn.Linear(in_features=64, out_features=2)
        self.flatten = nn.Flatten()
    def forward(self, x):
        x = self.flatten(x)
        
        x = self.fc1(x)
        x = nn.functional.leaky_relu(x)
        x = self.fc2(x)
        x = nn.functional.leaky_relu(x)
        x = self.fc3(x)
        x = nn.functional.leaky_relu(x)
        output = self.fc4(x)
        # output = nn.functional.softmax(x)
        return output

In [97]:
torch.cuda.set_device(0)
device = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")
print("device", torch.cuda.current_device(), torch.cuda.get_device_name(torch.cuda.current_device()))




net = Net()
net.to(device)

optimizer = torch.optim.Adam(net.parameters(), lr=0.002)
criterion  = nn.CrossEntropyLoss()


transforms_ = transforms.Compose([
    transforms.ToTensor(),
    transforms.Normalize([0.5], [0.5])
    ])
data_list = ['normal', 'bad_leg']
mydataset = MyDataset(data_list, transforms_)
train_len = int(len(mydataset)*0.8)
test_len = len(mydataset) - train_len
train_set, test_set = torch.utils.data.random_split(mydataset, [train_len, test_len])


train_loader = DataLoader(
    train_set,
    batch_size=64,
    shuffle=True,
    pin_memory=True
)
test_loader = DataLoader(
    test_set,
    batch_size=64,
    shuffle=True,
    pin_memory=True
)


device 0 NVIDIA GeForce RTX 2080 with Max-Q Design
[644.0021    449.46243     0.9781028]


In [98]:
training_start = time.time()
for epoch in range(1,50):
    loss_all = 0
    net.train()
    for i, batch in enumerate(train_loader):
        
        
        data = batch[0].to(device)
        label = batch[1].to(device)
        optimizer.zero_grad()
        
        output = net(data).squeeze()
        #output = output.topk(1)[1].squeeze()
        
        loss = criterion(output, label.long())
        loss.backward()
        loss_all += loss.item()
        optimizer.step()
        
        print(f'\rloss: {loss.item()}',end='')
        #print('\r',loss_all/(i+epoch*len(dataloader)))

    acc = 0
    for i, batch in enumerate(test_loader):
        
        net.eval()
        data = batch[0].to(device)
        label = batch[1].to(device)
        
        output = net(data).squeeze()
        #print(output)
        #output = output.topk(1)[1].squeeze()
        acc += np.array((torch.argmax(output, dim=1) == label.int()).cpu()).mean()
        
    print(' acc: ',acc/(i+1))
        
        

       


loss: 0.8216050267219543 acc:  0.5185185185185185
loss: 0.2955153286457062 acc:  0.8149305555555555
loss: 0.11084799468517303 acc:  0.9527777777777778
loss: 0.14201174676418304 acc:  0.9707175925925926
loss: 0.07996608316898346 acc:  0.9707175925925926
loss: 0.15369637310504913 acc:  0.9738425925925925
loss: 0.2348400503396988 acc:  0.9371527777777778
loss: 0.030021093785762787 acc:  0.9769675925925926
loss: 0.236335217952728273 acc:  0.9707175925925926
loss: 0.6275307536125183 acc:  0.9582175925925925
loss: 0.11422302573919296 acc:  0.9601851851851851
loss: 0.0074761416763067245 acc:  0.978125
loss: 0.04473799839615822 acc:  0.984375
loss: 0.10187452286481857 acc:  0.975
loss: 0.019047854468226433 acc:  0.984375
loss: 0.01747709885239601 acc:  0.9875
loss: 0.030268928036093712 acc:  0.9769675925925926
loss: 0.1794118583202362 acc:  0.984375
loss: 0.3320147693157196 acc:  0.9769675925925926
loss: 0.011837219819426537 acc:  0.9738425925925925
loss: 0.003259862307459116 acc:  0.986342592

In [94]:
acc = 0
for i, batch in enumerate(test_loader):
    
    net.eval()
    data = batch[0].to(device)
    label = batch[1].to(device)
    
    output = net(data).squeeze()
    #print(output)
    #output = output.topk(1)[1].squeeze()
    print(torch.argmax(output, dim=1))
    print(label.int())
    print((torch.argmax(output, dim=1) == label.int()))
    acc += np.array((torch.argmax(output, dim=1) == label.int()).cpu()).mean()
    
print(acc/(i+1))
        

       

tensor([1, 1, 1, 1, 1, 0, 1, 1, 1, 1, 0, 1, 0, 0, 0, 1, 0, 1, 1, 1, 1, 0, 1, 0,
        0, 1, 1, 1, 1, 1, 0, 1, 1, 1, 0, 1, 1, 1, 1, 1, 0, 0, 0, 1, 1, 0, 1, 1,
        0, 1, 1, 1, 1, 1, 0, 0, 1, 0, 0, 0, 1, 1, 0, 1], device='cuda:0')
tensor([1, 1, 1, 1, 1, 0, 1, 1, 1, 1, 0, 1, 0, 0, 0, 1, 0, 1, 1, 1, 1, 0, 1, 0,
        0, 1, 1, 1, 1, 1, 0, 1, 1, 1, 0, 1, 1, 1, 1, 1, 0, 0, 0, 1, 1, 0, 1, 1,
        0, 1, 1, 0, 1, 1, 0, 1, 1, 0, 0, 0, 1, 1, 0, 1], device='cuda:0',
       dtype=torch.int32)
tensor([ True,  True,  True,  True,  True,  True,  True,  True,  True,  True,
         True,  True,  True,  True,  True,  True,  True,  True,  True,  True,
         True,  True,  True,  True,  True,  True,  True,  True,  True,  True,
         True,  True,  True,  True,  True,  True,  True,  True,  True,  True,
         True,  True,  True,  True,  True,  True,  True,  True,  True,  True,
         True, False,  True,  True,  True, False,  True,  True,  True,  True,
         True,  True,  True,  True], d

In [89]:
a = np.array([1,2,3])
b = np.array([1,1,1])

print((a == b).mean())

0.3333333333333333
